In [1]:
import os
import sys
import glob

sys.path.append("Hand_pose_estimation_3D/arm_and_hand")
sys.path.append("Hand_pose_estimation_3D")

from dataloader_ann import HandArmLandmarksDataset_ANN

import numpy as np
from torch.utils.data import DataLoader
import os
from datetime import datetime
import math
import open3d as o3d
from utilities import convert_to_left_shoulder_coord
import time
import pandas as pd
from csv_writer import fusion_csv_columns_name
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import threading
import queue

from leftarm_angle_calculation import calculate_six_arm_angles, rotation_matrix_for_elbow, rotation_matrix_for_wrist
from lefthand_angle_calculation import FingerAngles
from send_leftarm_data_to_robot import send_leftarm_angles_udp_message_using_pid

from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATA_DIR = "data"  
SELECTED_DATE = "*"  # Keep '*' when using glob.glob

train_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "train")))
val_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "val")))

train_paths = ["data/2024-09-30/2024-09-30-17:09/landmarks_all_2024-09-30-17:09.csv"]

#fake_train_paths = glob.glob(os.path.join(DATA_DIR, "fake_data", "train", "fake_*.csv"))

#if len(fake_train_paths) > 0:
    #train_paths.extend(fake_train_paths)

body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]
train_body_distance_thres = 550
train_leftarm_distance_thres = 550
train_lefthand_distance_thres = 200
val_body_distance_thres = 450
val_leftarm_distance_thres = 450
val_lefthand_distance_thres = 150

train_dataset = HandArmLandmarksDataset_ANN(train_paths, 
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataset = HandArmLandmarksDataset_ANN(val_paths,
    body_lines,
    lefthand_lines,
    val_body_distance_thres,
    val_leftarm_distance_thres,
    val_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)  

In [3]:
arm_hand_fused_names = ["left shoulder", "left elbow", "left hip", "right shoulder", "right hip", 
 "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", "INDEX_FINGER_MCP", 
 "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", 
 "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", 
 "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", 
 "PINKY_DIP", "PINKY_TIP", "right elbow"]

In [4]:
target_output = train_dataset._outputs  #  shape: (N, 144)
sample = target_output[10]  # shape: (144)
sample = sample.reshape(3, 48)  # shape: (3, 48)
sample = sample.T  # shape: (48, 3)
sample = sample[:26]  # shape: (26, 3)

In [5]:
body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]
lines = body_lines.copy()
lines.extend(lefthand_lines)

In [6]:
pts, original_xyz = convert_to_left_shoulder_coord(sample, arm_hand_fused_names)

In [7]:
lefthand_angles = FingerAngles(arm_hand_fused_names)

In [8]:
wrist_to_home_position_thumb_finger_quat = [-0.794, 0.078, 0.105, 0.593]
wrist_to_home_position_index_finger_quat = [0.595, 0.517, 0.465, 0.404]
wrist_to_home_position_middle_finger_quat = [0.534, 0.508, 0.489, 0.466]
wrist_to_home_position_ring_finger_quat = [0.466, 0.489, 0.508, 0.534]
wrist_to_home_position_pinky_finger_quat = [0.404, 0.465, 0.517, 0.595]

j6_of_real_people_to_j6_in_rviz_rot_mat = R.from_euler("xz", [90, 180], degrees=True).as_matrix()
rot_mat_to_rearrange_finger_coord = R.from_euler("x", 90, degrees=True).as_matrix()

wrist_to_home_position_finger_quat_list = [
    wrist_to_home_position_thumb_finger_quat,
    wrist_to_home_position_index_finger_quat,
    wrist_to_home_position_middle_finger_quat,
    wrist_to_home_position_ring_finger_quat,
    wrist_to_home_position_pinky_finger_quat
]

In [9]:
colors = [[0, 0, 0] for i in range(len(lines))]
scale_factor = 10

angles, rot_mats_wrt_origin, rot_mats_wrt_parent = calculate_six_arm_angles(pts,
        original_xyz,
        arm_hand_fused_names)
angle_j1, angle_j2, angle_j3, angle_j4, angle_j5, angle_j6 = angles
j1_rm, j2_rm, j3_rm, j4_rm, j5_rm, j6_rm = rot_mats_wrt_origin

leftfingers_angle = lefthand_angles(pts, j6_rm)

Draw left arm

In [10]:
# Draw original xyz
x_unit, y_unit, z_unit = original_xyz[:, 0], original_xyz[:, 1], original_xyz[:, 2]
pts = np.concatenate([pts, [x_unit * scale_factor, y_unit * scale_factor, z_unit * scale_factor]], axis=0)
last_index = pts.shape[0] - 1
lines.extend([[0, last_index - 2], [0, last_index - 1], [0, last_index]])
colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Plot j1
x_j1, y_j1, z_j1 = j1_rm[:, 0],  j1_rm[:, 1], j1_rm[:, 2]
pts = np.concatenate([pts, [x_j1 * scale_factor, y_j1 * scale_factor, z_j1 * scale_factor]], axis=0)
last_index = pts.shape[0] - 1
lines.extend([[0, last_index - 2], [0, last_index - 1], [0, last_index]])
colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Plot j2
#x_j2, y_j2, z_j2 = j2_rm[:, 0],  j2_rm[:, 1], j2_rm[:, 2]
#pts = np.concatenate([pts, [x_j2 * scale_factor, y_j2 * scale_factor, z_j2 * scale_factor]], axis=0)
#last_index = pts.shape[0] - 1
#lines.extend([[0, last_index - 2], [0, last_index - 1], [0, last_index]])
#colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])


# Plot j3
elbow_idx = arm_hand_fused_names.index("left elbow")
elbow_landmark = pts[elbow_idx].copy()

j2_p_rm = j2_rm @ rotation_matrix_for_elbow 
x_j2_p_rm, y_j2_p_rm, z_j2_p_rm = j2_p_rm[:, 0], j2_p_rm[:, 1], j2_p_rm[:, 2]
x_j2_p_rm = x_j2_p_rm * scale_factor + elbow_landmark
y_j2_p_rm = y_j2_p_rm * scale_factor + elbow_landmark
z_j2_p_rm = z_j2_p_rm * scale_factor + elbow_landmark

pts = np.concatenate([pts, [x_j2_p_rm, y_j2_p_rm, z_j2_p_rm]], axis=0)
last_index = pts.shape[0] - 1
lines.extend([[elbow_idx, last_index - 2], [elbow_idx, last_index - 1], [elbow_idx, last_index]])
colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

x_j3, y_j3, z_j3 = j3_rm[:, 0],  j3_rm[:, 1], j3_rm[:, 2]
x_j3 = x_j3 * scale_factor + elbow_landmark
y_j3 = y_j3 * scale_factor + elbow_landmark
z_j3 = z_j3 * scale_factor + elbow_landmark

pts = np.concatenate([pts, [x_j3, y_j3, z_j3]], axis=0)
last_index = pts.shape[0] - 1
lines.extend([[elbow_idx, last_index - 2], [elbow_idx, last_index - 1], [elbow_idx, last_index]])
colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Plot j4
elbow_idx = arm_hand_fused_names.index("left elbow")
elbow_landmark = pts[elbow_idx].copy()

x_j4, y_j4, z_j4 = j4_rm[:, 0],  j4_rm[:, 1], j4_rm[:, 2]
x_j4 = x_j4 * scale_factor + elbow_landmark
y_j4 = y_j4 * scale_factor + elbow_landmark
z_j4 = z_j4 * scale_factor + elbow_landmark

#pts = np.concatenate([pts, [x_j4, y_j4, z_j4]], axis=0)
#last_index = pts.shape[0] - 1
#lines.extend([[elbow_idx, last_index - 2], [elbow_idx, last_index - 1], [elbow_idx, last_index]])
#colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Plot j5
wrist_idx = arm_hand_fused_names.index("WRIST")
wrist_landmark = pts[wrist_idx].copy()

j4_p_rm = j4_rm @ rotation_matrix_for_wrist 
x_j4_p_rm, y_j4_p_rm, z_j4_p_rm = j4_p_rm[:, 0], j4_p_rm[:, 1], j4_p_rm[:, 2]
x_j4_p_rm = x_j4_p_rm * scale_factor + wrist_landmark
y_j4_p_rm = y_j4_p_rm * scale_factor + wrist_landmark
z_j4_p_rm = z_j4_p_rm * scale_factor + wrist_landmark

pts = np.concatenate([pts, [x_j4_p_rm, y_j4_p_rm, z_j4_p_rm]], axis=0)
last_index = pts.shape[0] - 1
lines.extend([[wrist_idx, last_index - 2], [wrist_idx, last_index - 1], [wrist_idx, last_index]])
colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

x_j5, y_j5, z_j5 = j5_rm[:, 0],  j5_rm[:, 1], j5_rm[:, 2]
x_j5 = x_j5 * scale_factor + wrist_landmark
y_j5 = y_j5 * scale_factor + wrist_landmark 
z_j5 = z_j5 * scale_factor + wrist_landmark 

pts = np.concatenate([pts, [x_j5, y_j5, z_j5]], axis=0)
last_index = pts.shape[0] - 1
lines.extend([[wrist_idx, last_index - 2], [wrist_idx, last_index - 1], [wrist_idx, last_index]])
colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Plot j6
wrist_idx = arm_hand_fused_names.index("WRIST")
wrist_landmark = pts[wrist_idx].copy()

x_j6, y_j6, z_j6 = j6_rm[:, 0],  j6_rm[:, 1], j6_rm[:, 2]
x_j6 = x_j6 * scale_factor + wrist_landmark
y_j6 = y_j6 * scale_factor + wrist_landmark
z_j6 = z_j6 * scale_factor + wrist_landmark

#pts = np.concatenate([pts, [x_j6, y_j6, z_j6]], axis=0)
#last_index = pts.shape[0] - 1
#lines.extend([[wrist_idx, last_index - 2], [wrist_idx, last_index - 1], [wrist_idx, last_index]])
#colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

Draw left hand

In [11]:
#thumb_cmc_idx = arm_hand_fused_names.index("THUMB_CMC")
#thumb_cmc_landmark = pts[thumb_cmc_idx].copy()

#rot_mat_to_align_left_wrist_to_thumb = R.from_quat(wrist_to_home_position_thumb_finger_quat).as_matrix()
#thumb_home_coordinate = j6_rm @ j6_of_real_people_to_j6_in_rviz_rot_mat @ rot_mat_to_align_left_wrist_to_thumb

#thumb_x_j1, thumb_y_j1, thumb_z_j1 = thumb_home_coordinate[:, 0], thumb_home_coordinate[:, 1], thumb_home_coordinate[:, 2]
#thumb_x_j1 = thumb_x_j1 * scale_factor + thumb_cmc_landmark
#thumb_y_j1 = thumb_y_j1 * scale_factor + thumb_cmc_landmark
#thumb_z_j1 = thumb_z_j1 * scale_factor + thumb_cmc_landmark

#pts = np.concatenate([pts, [thumb_x_j1, thumb_y_j1, thumb_z_j1]], axis=0)
#last_index = pts.shape[0] - 1
#lines.extend([[thumb_cmc_idx, last_index - 2], [thumb_cmc_idx, last_index - 1], [thumb_cmc_idx, last_index]])
#colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [12]:
for i, finger_name in enumerate(lefthand_angles.fingers_name):
    finger_i_result = leftfingers_angle[finger_name]
    finger_i_angle_j1, finger_i_angle_j2, finger_i_angle_j3 = finger_i_result["angles"]
    finger_i_j1_rot_mat, finger_i_j2_rot_mat, finger_i_j3_rot_mat = finger_i_result["coordinates_wrt_origin"]

    if finger_name == "THUMB":
        mcp_name = f"{finger_name}_MCP"
        pip_name = f"{finger_name}_IP"
        dip_name = f"{finger_name}_TIP"
    else:
        if finger_name in ["INDEX", "MIDDLE", "RING"]:
            finger_name = f"{finger_name}_FINGER"
        mcp_name = f"{finger_name}_MCP"
        pip_name = f"{finger_name}_PIP"
        dip_name = f"{finger_name}_DIP"

    mcp_idx = arm_hand_fused_names.index(mcp_name)
    finger_i_mcp_landmark = pts[mcp_idx].copy()

    align_rot_mat = R.from_quat(wrist_to_home_position_finger_quat_list[i]).as_matrix()
    j6_p_rm = j6_rm @ j6_of_real_people_to_j6_in_rviz_rot_mat @ align_rot_mat @ rot_mat_to_rearrange_finger_coord
    x_j6_p_rm, y_j6_p_rm, z_j6_p_rm = j6_p_rm[:, 0], j6_p_rm[:, 1], j6_p_rm[:, 2]
    x_j6_p_rm = x_j6_p_rm * scale_factor + finger_i_mcp_landmark
    y_j6_p_rm = y_j6_p_rm * scale_factor + finger_i_mcp_landmark
    z_j6_p_rm = z_j6_p_rm * scale_factor + finger_i_mcp_landmark

    pts = np.concatenate([pts, [x_j6_p_rm, y_j6_p_rm, z_j6_p_rm]], axis=0)
    last_index = pts.shape[0] - 1
    lines.extend([[mcp_idx, last_index - 2], [mcp_idx, last_index - 1], [mcp_idx, last_index]])
    colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

    #x_finger_i_j1, y_finger_i_j1, z_finger_i_j1 = finger_i_j1_rot_mat[:, 0], finger_i_j1_rot_mat[:, 1], finger_i_j1_rot_mat[:, 2]
    #x_finger_i_j1 = x_finger_i_j1 * scale_factor + finger_i_mcp_landmark
    #y_finger_i_j1 = y_finger_i_j1 * scale_factor + finger_i_mcp_landmark
    #z_finger_i_j1 = z_finger_i_j1 * scale_factor + finger_i_mcp_landmark

    #pts = np.concatenate([pts, [x_finger_i_j1, y_finger_i_j1, z_finger_i_j1]], axis=0)
    #last_index = pts.shape[0] - 1
    #lines.extend([[mcp_idx, last_index - 2], [mcp_idx, last_index - 1], [mcp_idx, last_index]])
    #colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

    x_finger_i_j2, y_finger_i_j2, z_finger_i_j2 = finger_i_j2_rot_mat[:, 0], finger_i_j2_rot_mat[:, 1], finger_i_j2_rot_mat[:, 2]
    x_finger_i_j2 = x_finger_i_j2 * scale_factor + finger_i_mcp_landmark
    y_finger_i_j2 = y_finger_i_j2 * scale_factor + finger_i_mcp_landmark
    z_finger_i_j2 = z_finger_i_j2 * scale_factor + finger_i_mcp_landmark

    pts = np.concatenate([pts, [x_finger_i_j2, y_finger_i_j2, z_finger_i_j2]], axis=0)
    last_index = pts.shape[0] - 1
    lines.extend([[mcp_idx, last_index - 2], [mcp_idx, last_index - 1], [mcp_idx, last_index]])
    colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

    # Plot j2 -j3 of fingers
    if finger_i_j3_rot_mat is not None:
        pip_idx = arm_hand_fused_names.index(pip_name)
        finger_i_pip_landmark = pts[pip_idx].copy()

        finger_i_j1_p_rm = finger_i_j1_rot_mat @ lefthand_angles._rot_mat_for_fingers_j3
        x_finger_i_j1_p_rm, y_finger_i_j1_p_rm, z_finger_i_j1_p_rm = finger_i_j1_p_rm[:, 0], finger_i_j1_p_rm[:, 1], finger_i_j1_p_rm[:, 2]
        x_finger_i_j1_p_rm = x_finger_i_j1_p_rm * scale_factor + finger_i_pip_landmark
        y_finger_i_j1_p_rm = y_finger_i_j1_p_rm * scale_factor + finger_i_pip_landmark
        z_finger_i_j1_p_rm = z_finger_i_j1_p_rm * scale_factor + finger_i_pip_landmark

        pts = np.concatenate([pts, [x_finger_i_j1_p_rm, y_finger_i_j1_p_rm, z_finger_i_j1_p_rm]], axis=0)
        last_index = pts.shape[0] - 1
        lines.extend([[pip_idx, last_index - 2], [pip_idx, last_index - 1], [pip_idx, last_index]])
        colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

        x_finger_i_j3, y_finger_i_j3, z_finger_i_j3 = finger_i_j3_rot_mat[:, 0], finger_i_j3_rot_mat[:, 1], finger_i_j3_rot_mat[:, 2]
        x_finger_i_j3 = x_finger_i_j3 * scale_factor + finger_i_pip_landmark
        y_finger_i_j3 = y_finger_i_j3 * scale_factor + finger_i_pip_landmark
        z_finger_i_j3 = z_finger_i_j3 * scale_factor + finger_i_pip_landmark

        pts = np.concatenate([pts, [x_finger_i_j3, y_finger_i_j3, z_finger_i_j3]], axis=0)
        last_index = pts.shape[0] - 1
        lines.extend([[pip_idx, last_index - 2], [pip_idx, last_index - 1], [pip_idx, last_index]])
        colors.extend([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [13]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pts)

line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(pts),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd, line_set])